<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [0]:
# Question1

#Yesss I fixed that problem

host_url = ["https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"]
load_more_urls = ["https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crmqm2dczaa7wxhfmzzrdtmsbzhzfmxvlnomwklyczuf43o6ss6oe2fjmzldj4k5tm5fjyr3a2pn6woclsokk47cma", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cjmqqzde3yk7swhhmjqrxt4ohjjtzpwzouokkd2gbzgpnt6ucc2peyv5mbjb4dwdpdqzcxjfmg3k65z4tqdaz44w", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbkqmzdizyc76wxvnzrrxs46bzhzfmxvlnomwklyczuf43o6ss6oe4vtpridv4k5vz4e43vvsqoskmtj4psbfq2qni", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbpryzd6yab62tx7obtrtt4objhzfmxvlnomwklyczuf43o6ss6oe4f5nzldn4k4k7d72vy22axhza7lis2c2y7jzi"]
all_urls = host_url+load_more_urls
user_names, user_links, content, date, title, review_helpfulness = [],[], [], [], [], []
review_elements = []
for url in all_urls:
  request = urllib.request.urlopen (url)
  webpage  = request.read()
  soup = BeautifulSoup(webpage)

  reviews = soup.find_all("div", {"class":"lister-item-content" })
  for review in reviews:
    try:
        review_title = review.find("a", class_="title").text.strip()
    except:
        review_title = ""
    try:
        review_id = review.find("a")["href"].split("/")[2]
    except:
        review_id = ""
    try:
        review_date = review.find("span", class_="review-date").get_text()
    except:
        review_date = ""
    try:
        review_nickname = review.find ("span", class_="display-name-link").get_text()
    except:
        review_nickname = ""
    try:
        review_content = review.find ("div", class_="text show-more__control").get_text()
    except:
        review_content =""
    #print(review_content)
    try:
        review_rating = review. find ("span", class_="rating-other-user-rating").getText().strip()
        #print(review_rating)
    except:
        review_rating = ""
    try:
        review_helpfulness = review.find("div", {"class": "actions text-muted"}).text.split("\n")[1]
        #print(review_helpfulness)
    except:
        review_helpfulness = ""

    review_elements.append([review_title, review_id, review_date, review_nickname, review_content, review_rating, review_helpfulness])
#print(len(review_elements))

df = pd.DataFrame(review_elements, columns=["review_title", "review_id", "review_date", "review_nickname", "review_content", "review_rating", "review_helpfulness"])
df

with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "w", newline="", encoding='utf-8') as file:
  df.to_csv(file)
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r", encoding='utf-8') as file:
  pd.read_csv(file)



In [0]:
#try_codes
load_more_data = soup.find_all("div", attrs={"class": "load-more-data"}) # find this button at the end of each page
#print(load_more_data)
for item in load_more_data:
  path_parse1 = "https://www.imdb.com"
  path_parse2 = "/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey="
  path_parse3 = item.get("data-key")
  concatenated_path = path_parse1+path_parse2+path_parse3
  print(concatenated_path)
#load_more_links = ["https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crmqm2dczaa7wxhfmzzrdtmsbzhzfmxvlnomwklyczuf43o6ss6oe2fjmzldj4k5tm5fjyr3a2pn6woclsokk47cma", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cjmqqzde3yk7swhhmjqrxt4ohjjtzpwzouokkd2gbzgpnt6ucc2peyv5mbjb4dwdpdqzcxjfmg3k65z4tqdaz44w", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbkqmzdizyc76wxvnzrrxs46bzhzfmxvlnomwklyczuf43o6ss6oe4vtpridv4k5vz4e43vvsqoskmtj4psbfq2qni", "https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbpryzd6yab62tx7obtrtt4objhzfmxvlnomwklyczuf43o6ss6oe4f5nzldn4k4k7d72vy22axhza7lis2c2y7jzi"]
#for link in load_more_links:
 # print(link) 

https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7crmqu2ta3qb7oux7nrvqpr42brhzfmxvlnomwklyczuf43o6ss6oe2fjmzldj4k4rmwb65sqjd2c5p4ic5kj7njmla


In [0]:
# In this line, I am attempting to generate links with data-key in the class load-more-data. However, because, the server prohibits to get key in the second next page, so my code raise error.
host_url = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"
def link_load_more (host_url):
  request = urllib.request.urlopen (host_url)
  webpage  = request.read()
  soup = BeautifulSoup(webpage)
  load_more_data = soup.find_all("div", attrs={"class": "load-more-data"}) # find this button at the end of each page
  flag = True
  if len (load_more_data):
  #print(load_more_data)
    for item in load_more_data:
      path_parse1 = "https://www.imdb.com/"
      path_parse2 = "title/tt7286456/reviews/_ajax"
      path_parse3 = "?ref_=undefined&paginationKey="
      try:
        path_parse4 = item.get("data-key")
        concatenated_path = path_parse1+path_parse2+path_parse3+path_parse4
      except KeyError:
        flag = False
  return concatenated_path

# generate link and parse all of four pages

url_nextpage1 = link_load_more(host_url)
url_nextpage2 = link_load_more(url_nextpage1)
url_nextpage3 = link_load_more(url_nextpage2)
url_nextpage4 = link_load_more(url_nextpage3)

print(url_nextpage1)
print(url_nextpage2)
print(url_nextpage3)
print(url_nextpage4)



NameError: ignored

In [0]:
from bs4 import BeautifulSoup
import requests

session = requests.Session()

link = 'https://www.imdb.com/title/tt7286456/reviews'
r=requests.get(link)
r.request.headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [0]:
from bs4 import BeautifulSoup
import requests

session = requests.Session()

link = 'https://www.imdb.com/title/tt7286456/reviews'
UA = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'close',
    'Upgrade-Insecure-Requests': '1'
}

getlink = session.get(link, headers=UA)
resplink = BeautifulSoup(getlink.content, 'html.parser')
key = resplink.find('div', {'class':'load-more-data'})
datakey = key.attrs['data-key']

UAloadmore = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'Accept': '*/*',
    'Accept-Language': 'en',
    'Accept-Encoding': 'gzip, deflate',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'close',
    'Referer': 'https://www.imdb.com/title/tt7286456/reviews'
}

a = True
while a:
    if key is None:
        print('Loi')
        a = False
    else:
        linkloadmore = 'https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey='+datakey
        getloadmore = session.get(linkloadmore, headers=UAloadmore)
        resploadmore = BeautifulSoup(getloadmore.content, 'html.parser')
        key = resploadmore.find('div', {'class':'load-more-data'})
        datakey = key.attrs['data-key']
        print('https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey='+datakey)

https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cjmq45deyqd72xh5nrrrhrm4azhzfmxvlnomwklyczuf43o6ss6oe4f3njmdz4k4da6ioyalzd4xa252qu5ftukbqy
https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbkry3dkzaf62vhtojxrxs4oab5y4hhzo5ziwr26fbyhvrl4ty4ouyflpzidvwndtuwrturx3vuipk6vekqxnoducax
https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbiq45dozib7cwxfnrqrlumsbbhzfmxvlnomwklyczuf43o6ss6oizvjmbpdr4k5bqann64snddzfb2neeojfggo6i
https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cboqe3t6yac7stx5obsrhrmqabyy4hhzo5ziwr26fbyhvrl4ty4ouyftmrpdjwndtvdmtgkb2h76sxkaxt4g3rbseru
https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbnry5di3yh7kwxhmrrrdtmwhjjtzpwzouokkd2gbzgpnt6ucc2pa4vxnjib4d7boidprjzxgrueniiccsyawgw6
https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbmqa3d4yqd7ktx3njzrts4obrhzfmxvlno

KeyboardInterrupt: ignored

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [0]:
# Write your code here
#clean the text data
from __future__ import division
import nltk # re, pprint
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string 

with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r", encoding='utf-8') as file:
  data = pd.read_csv(file)

raw_title = [str(line) for line in data.review_title]
raw_content = [str(line) for line in data.review_content]
# Tokenize raw text in raw_title and raw_content
tokens_title= [nltk.word_tokenize(title) for title in raw_title]
tokens_content = [nltk.word_tokenize(item) for item in raw_content]

# PROCCESSES OF CLEANING TEXT  
def cleaning_text (text):
    step1 = remove_noise(text)
    step2 = remove_num (step1)
    step3 = remove_stop (step2)
    step4 = lowercase (step3)
    step5 = stemming (step4)
    step6 = lemma (step5)
    return step6

# (1) Remove noise, such as special characters and punctuations.      
def remove_noise (list):
    punct=[char for char in string.punctuation]
    noise_removed = [word for word in list if word not in punct]
    return noise_removed

#(2) Remove numbers.
def remove_num (list):
    num_removed  = [word for word in list if not word.isnumeric()]
    return num_removed

 # (3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).
def remove_stop (list):
    stopword_removed = [word for word in list if word not in stopwords.words('english')]
    return stopword_removed

#(4) Lowercase all texts
def lowercase (list):
    lowercase = [word.lower() for word in list]
    return lowercase
# (5) Stemming
def stemming (list):
    ps = PorterStemmer()
    word_stemmed = [ps.stem(word) for word in list]
    return word_stemmed
# (6) Lemmatization.
def lemma (list):
    wnl =  WordNetLemmatizer ()
    word_lemmatized = [wnl.lemmatize(word) for word in list]
    return word_lemmatized

# Cleaning title reviews and content review
cleaned_title = [cleaning_text (sublist) for sublist in tokens_title]
cleaned_content = [cleaning_text (sublist) for sublist in tokens_content]
# Save cleaned text into new columns in dataframe
df["cleaned_title"] = cleaned_title
df["cleaned_content"] = cleaned_content
df






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,review_title,review_id,review_date,review_nickname,review_content,review_rating,review_helpfulness,cleaned_title,cleaned_content
0,As a viewer that actually went to TIFF and wit...,rw5112402,10 September 2019,JF500,I was a person that saw all the hype and claim...,10/10,"6,840 out of 7,862 found t...","[a, viewer, actual, went, tiff, wit, film, n't...","[i, person, saw, hype, claim, masterpiec, over..."
1,Outstanding movie with a haunting performance ...,rw5159304,3 October 2019,MihaVrhunc,"Every once in a while a movie comes, that trul...",10/10,"3,375 out of 4,039 found t...","[outstand, movi, haunt, perform, best, charact...","[everi, movi, come, truli, make, impact, joaqu..."
2,Only certain people can relate,rw5168360,7 October 2019,lesterarnoldpinto,This is a movie that only those who have felt ...,10/10,"2,676 out of 3,304 found t...","[onli, certain, peopl, relat]","[thi, movi, felt, alon, isol, truli, relat, yo..."
3,Must have put a SMILE of Satisfaction on Heath...,rw5092831,1 September 2019,Chandler_Bing_,"Truly a masterpiece, The Best film of 2019, on...",10/10,"3,166 out of 3,935 found t...","[must, put, smile, satisfact, heath, ledger, '...","[truli, masterpiec, the, best, film, one, best..."
4,The Hype is real,rw5160204,4 October 2019,kdagoulis26,Most of the time movies are anticipated like t...,10/10,"2,352 out of 2,918 found t...","[the, hype, real]","[most, time, movi, anticip, like, end, fall, s..."
...,...,...,...,...,...,...,...,...,...
109,"Spare me the ""best comic book movie ever"" nons...",rw5144576,26 September 2019,gatozangado,It premiered last night in Sydney and I was lu...,10/10,102 out of 206 found this ...,"[spare, ``, best, comic, book, movi, ever, '',...","[it, premier, last, night, sydney, i, lucki, e..."
110,Don't fall for the hype,rw5206952,23 October 2019,ninjashidan,I saw the joker last night and was highly disa...,1/10,88 out of 176 found this h...,"[do, n't, fall, hype]","[i, saw, joker, last, night, highli, disappoin..."
111,As movie well overrated - acting excellent,rw5175282,9 October 2019,dhllon,The depth of the story was lacking to much foc...,2/10,59 out of 114 found this h...,"[a, movi, well, overr, act, excel]","[the, depth, stori, lack, much, focu, short, a..."
112,Lives up to the hype,rw5158293,3 October 2019,i_like_jellycups,A breath of fresh airThink heath ledgers joker...,10/10,99 out of 200 found this h...,"[live, hype]","[a, breath, fresh, airthink, heath, ledger, jo..."


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [0]:
# Write your code here
#(1) Tag parts of Speech (POS) and  calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb)...
nltk.download('averaged_perceptron_tagger')
from collections import Counter
count_POStag = []
for tokens in cleaned_title:
    title_tagged = nltk.pos_tag(tokens)
    #fd_title = nltk.FreqDist(tag for (token, tag) in title_tagged)
    #print(fd_title)
    count_POStag.append(Counter (tag for tokens, tag in title_tagged))
for tokens in cleaned_content:
    content_tagged = nltk.pos_tag(tokens)
    #fd_title = nltk.FreqDist(tag for (token, tag) in title_tagged)
    #print(fd_title)
    count_POStag. append (Counter (tag for tokens, tag in content_tagged))
    
print(len(count_POStag))
print(count_POStag)
count_VB=0
count_NN=0
count_ADJ=0
count_ADV=0
for pack in count_POStag:
  for item in pack:
    if item =="VB" or item=="VBG" or item=="VBD" or item=="VBP" or item=="VBZ":
      count_VB+=1
    elif item =="NN" or item=="NNS" or item =="NNP" or item=="NNPS":
      count_NN+=1
    elif item =="JJ" or item =="JJR" or item=="JJS":
      count_ADJ+=1
    elif item =="RB" or item=="RBR" or item=="RBS":
      count_ADV+=1
    else:
      pass
print("The total verbs:", count_VB)
print("The total nouns:", count_NN)
print("The total adjectives:", count_ADJ)
print("The total adverbs:", count_ADV)
#for n in count_POStag:
 # for items in count_POStag
 # hist[key]=hist.get(valu=)+1
  




[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
228
[Counter({'NN': 9, 'JJ': 3, 'DT': 1, 'VBD': 1, 'RB': 1, 'VB': 1}), Counter({'NN': 5, 'JJS': 1, 'VB': 1, 'RB': 1, 'VBN': 1}), Counter({'NN': 3, 'JJ': 1}), Counter({'NN': 4, 'RB': 2, 'MD': 1, 'VB': 1, 'JJ': 1, 'POS': 1}), Counter({'DT': 1, 'NN': 1, 'JJ': 1}), Counter({'NN': 2}), Counter({'RB': 1, 'VB': 1, 'WRB': 1, 'JJ': 1, 'VBP': 1}), Counter({'NN': 2, 'VBD': 1, 'JJ': 1}), Counter({'NN': 3, 'DT': 1, 'RB': 1, 'VBD': 1}), Counter({'NN': 6, 'POS': 1, 'JJS': 1, 'VBZ': 1, ':': 1, 'VBD': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'JJ': 2, 'NN': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'NN': 4, 'RB': 2, 'VB': 1}), Counter({'NN': 2}), Counter({'JJ': 1, 'NN': 1}), Counter({'NN': 2}), Counter({'NN': 2}), Counter({'VB': 2, 'VBP': 1, 'RB': 1}), Counter({'NN': 2, 'MD': 1, 'VB': 1}), Counter({'NN': 3, 'CD': 1,

In [0]:
# (2)Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences.
import spacy
!pip install benepar
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')

In [70]:
# (2)Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences.
# Download and import necessarry packages
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')
!pip install benepar
from nltk.tree import Tree 
benepar.download('benepar_en2')
nltk.download('punkt')
import benepar


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
[nltk_data] Downloading package benepar_en2 to /root/nltk_data...
[nltk_data]   Package benepar_en2 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
('S',)
The time for action


In [58]:
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en'))
doc = nlp("The time for action is now. It's never too late to do something.")
sent = list(doc.sents)[0]
parse_sent = sent._.parse_string
# (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
print(sent._.labels)
# ('S',)
print(list(sent._.children)[0])
# The time for action


('S',)
The time for action


In [69]:
#Constituency Parsing
from nltk.tree import Tree 
benepar.download('benepar_en2')
nltk.download('punkt')
import benepar
parser = benepar.Parser("benepar_en2")
tree = parser.parse("Apple is looking at buying U.K. startup for $1 billion.")
tree.pretty_print()

[nltk_data] Downloading package benepar_en2 to /root/nltk_data...
[nltk_data]   Package benepar_en2 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                              S                                          
   ___________________________|________________________________________   
  |          VP                                                        | 
  |     _____|__________                                               |  
  |    |                VP                                             | 
  |    |      __________|________________                              |  
  |    |     |                           PP                            | 
  |    |     |      _____________________|_____                        |  
  |    |     |     |                           S                       | 
  |    |     |     |                           |                       |  
  |    |     |     | 

In [47]:
#dependence parsing tree
from spacy import displacy
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
  print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
displacy.render(doc, style = "dep", jupyter=True, options={"distance":140})

Apple Apple PROPN NNP nsubj Xxxxx True False
is be VERB VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [49]:
import spacy
from nltk import Tree


en_nlp = spacy.load('en')

doc = en_nlp("Apple is looking at buying U.K. startup for $1 billion.")

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

          looking                           
   __________|______________                 
  |    |     |              at              
  |    |     |              |                
  |    |     |            buying            
  |    |     |        ______|_______         
  |    |     |       |             for      
  |    |     |       |              |        
  |    |     |    startup        billion    
  |    |     |       |       _______|_____   
Apple  is    .      U.K.    $             1 



[None]

In [45]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
   print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [31]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
​
for token in doc:
  print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)


SyntaxError: ignored

SyntaxError: ignored

In [9]:
 import stanfordnlp
stanfordnlp.download('en')   # This downloads the English models for the neural pipeline
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English
doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0].print_dependencies()

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:10<00:00, 19.6MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources

/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [12]:
!pip install pycorenlp

  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-cp36-none-any.whl size=2143 sha256=96bf71a93af31cb4c4308d5b89c812b37a3b4c1f55666c67b0ff8db694b37b8f
  Stored in directory: /root/.cache/pip/wheels/fb/e9/2f/767a7b5f2e82d587a36143c04a21839b4b14bebfb89410d2d5
Successfully built pycorenlp


In [0]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [0]:
from nltk.parse import CoreNLPParser

result = CoreNLPParser(url='http://localhost:9000')

In [0]:
from pycorenlp import StanfordCoreNLP

nlp_wrapper = StanfordCoreNLP('http://localhost:9000')

In [21]:
doc = "Ronaldo has moved from Real Madrid to Juventus. While messi still plays for Barcelona"
nlp_wrapper = result.annotate(doc,
    properties={
        'annotators': 'ner, pos',
        'outputFormat': 'json',
        'timeout': 1000,
    })

AttributeError: ignored

In [16]:
#constitutuent Parsing with Standford NP
import nltk
from nltk.parse.stanford import StanfordParser
def traverse_phrase(tree): 

    for subtree in tree:

        if type(subtree) == nltk.tree.Tree:

            traverse_phrase(subtree)

        else:

            print( "constituent : " + subtree )

def traverse_tree(tree):

    for subtree in tree:

        if type(subtree) == nltk.tree.Tree:

            if subtree.label() == 'NP':

                print("\n[Noun Phrase]")

                traverse_phrase(subtree)

            else :

                traverse_tree(subtree)

stanford_parser_dir = "C:\\Users\ngoch\\stanford-parser-full-2015-04-20\\" 
path_to_models = stanford_parser_dir  + "stanford-parser-3.5.2-models.jar"
path_to_jar = stanford_parser_dir  + "stanford-parser.jar"
sentence = ("The cute cat chases the mouse")

# Phrase structure parser

parser=StanfordParser(path_to_models, path_to_jar)

print( "\n\nPhrase Structure Parsing Result - will only print NPs (Noun Phrases)")
parsed_sentence = parser.raw_parse(sentence)
traverse_tree(parsed_sentence)



LookupError: ignored

In [17]:
dependency_pattern = '{left}<---{word}[{w_type}]--->{right}\n--------'
sentence_nlp = ["Spacy had two types of English dependency parsers based on what language models you use"]
for token in sentence_nlp:
    print(dependency_pattern.format(word=token.orth_, 
                                  w_type=token.dep_,
                                  left=[t.orth_ 
                                            for t 
                                            in token.lefts],
                                  right=[t.orth_ 
                                             for t 
                                             in token.rights]))

AttributeError: ignored

In [0]:
sentence = str(news_df.iloc[1].full_text)
sentence_nlp = nlp(sentence)

# print named entities in article
print([(word, word.ent_type_) for word in sentence_nlp if word.ent_type_])

# visualize named entities
displacy.render(sentence_nlp, style='ent', jupyter=True)


NameError: ignored

In [0]:
import os
print("JAVA_HOME:", os.environ['JAVA_HOME']) 
java_path = r'C:\Program Files\Java\jdk1.8.0_102\bin\java.exe'
os.environ['JAVAHOME'] = java_path

from nltk.parse.stanford import StanfordParser
scp = StanfordParser(path_to_jar= r"C:\Users\ngoch\stanford-parser-full-2015-04-20\stanford-parser.jar",
                     path_to_models_jar= r"C:\Users\ngoch\stanford-parser-full-2015-04-20\stanford-parser-3.5.2-models.jar")
result = list(scp.raw_parse("I saw the joker last night"))
print(result[0])

KeyError: ignored

In [0]:
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)
print(grammar1)
for sent in cleaned_content:
  rd_parser = nltk.RecursiveDescentParser(grammar1)
  for tree in rd_parser.parse(sent):
     print(tree)

Grammar with 25 productions (start state = S)
    S -> NP VP
    VP -> V NP
    VP -> V NP PP
    PP -> P NP
    V -> 'saw'
    V -> 'ate'
    V -> 'walked'
    NP -> 'John'
    NP -> 'Mary'
    NP -> 'Bob'
    NP -> Det N
    NP -> Det N PP
    Det -> 'a'
    Det -> 'an'
    Det -> 'the'
    Det -> 'my'
    N -> 'man'
    N -> 'dog'
    N -> 'cat'
    N -> 'telescope'
    N -> 'park'
    P -> 'in'
    P -> 'on'
    P -> 'by'
    P -> 'with'


ValueError: ignored

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 